# Paths in Flat Triangulations

We will draw some paths in a surface and use it to define a deformation. As a simple example, consider the torus.

In [ ]:
from ipyvue_flatsurf import Widget

In [ ]:
from flatsurf import translation_surfaces
T = translation_surfaces.square_torus()
W = Widget(T)
W

Execute the following cell. Then draw a path on the above surface. The path may cross above half edges and does not have to be in a straight line but currently it must start and end at a vertex. Select the same point twice when you are done.

In [ ]:
W.path = None
path = await W.path

Internally, the path is represented as a homotopic sequence of saddle connections, in fact, as a sequence of half edges in the surface:

In [ ]:
print(path)

To make sense of these half edges in the picture above, we change the numbering of edges to the one used internally:

In [ ]:
W.labels = 'NUMERIC'

The sequence of half edges might be a bit complicated to understand. We can replace it by a tightened version of the path that does not contain any angles less than π and show that path in the picture above instead:

In [ ]:
tight = path.tighten()
print(tight)
W.action = "glue"
W.path = tight

## A more Interesting Surface
We will now do the same on a more interesting surface. The unfolding of the (3, 4, 13) triangle.

In [ ]:
from flatsurf import polygons, similarity_surfaces
t = polygons.triangle(3, 4, 13)
B = similarity_surfaces.billiard(t)
T = B.minimal_cover('translation')
T = T.erase_marked_points()

In [ ]:
from ipyvue_flatsurf import Widget
W = Widget(T)
W.labels = "NUMERIC"
W

Again, we will draw a path on the surface. Select the same point twice when you are done.

In [ ]:
W.path = None
W.action = None
W.action = "path"
path = await W.path
print(path)

We tighten the path and show the tigthened version instead.

In [ ]:
tight = path.tighten()
print(tight)
W.action = "glue"
W.path = tight

The path you drew might now consist of more than one saddle connection. Let us focus on the first saddle connection of the path:

In [ ]:
direction = next(iter(tight))
W.path = None
W.saddle_connections = [direction]

We can compute the flow decomposition in the direction defined by this connection:

In [ ]:
from flatsurf import GL2ROrbitClosure
O = GL2ROrbitClosure(T)
D = O.decomposition(direction.vector())
S = D.surface()
print(D)

We fix a random cylinder of this flow decomposition and highlight it in the above picture.

In [ ]:
cylinder = next(iter(component for component in D.components() if component.cylinder()))
W.flow_components = [cylinder]

Let us deform our surface by deforming this cylinder. If we want to deform the cylinder, we can equally deform all the edges the cylinder crosses. We deform the edges corresponding to how often they are crossed by the cylinder.
So, let us determine which half edges the cylinder crosses. We walk slightly left of the right side of the cylinder to determine which half edges we cross.

In [ ]:
right = list(step.saddleConnection() for step in cylinder.right())

crossed = []

for at, connection in enumerate(right):
    crossed.extend([p.halfEdge() for p in connection.path()])
    
    target_half_edge = connection.target()
    
    if connection != type(connection)(S, -target_half_edge):
        crossed.append(target_half_edge)
        
    while True:
        target_half_edge = S.previousAtVertex(target_half_edge)
        if target_half_edge == right[(at + 1) % len(right)].source(): break
        crossed.append(target_half_edge)
        
print(crossed)

We can now deform the cylinder. We write the deformation as a component in flow direction and one that is aligned with the base of the cylinder. To make the deformation visually appealing for this demo, we deform such that the cylinder gets about fives times as wide as it used to be:

In [ ]:
parallel = direction.vector()
nonparallel = list(cylinder.top())[0].saddleConnection().vector()

long = 1R
wide = 4R # Must be > -1.

deformation_vector = long * parallel + wide * nonparallel

We count how often each edge is being crossed to construct a deformation:

In [ ]:
crossings = {
    edge: crossed.count(edge.positive()) - crossed.count(edge.negative())
    for edge in S.edges()
}
print(crossings)

Now we deform the surface and display the resulting surface:

In [ ]:
deformation = S + [
    crossings[edge] * deformation_vector
    for edge in S.edges()
]

In [ ]:
from flatsurf.geometry.pyflatsurf_conversion import from_pyflatsurf
deformed = deformation.codomain()
WW = Widget(deformed)
WW

Probably, that picture is not terribly helpful as the triangulation might have been stretched out extremely in the process.

Anyway, we can reconstruct the deformed cylinder in the new surface since we can relate the half edges before and after the deformation:

In [ ]:
O = GL2ROrbitClosure(deformed)
D = O.decomposition(direction.vector())
print(D)

In [ ]:
cylinders = [component for component in D.components() if component.cylinder()]

In [ ]:
WW.flow_components = cylinders

In [ ]:
from pyflatsurf import flatsurf

Path = flatsurf.Path[type(deformed)]
SaddleConnection = type(next(iter(cylinder.right())).saddleConnection())

source = next(iter(cylinder.right())).saddleConnection().source()

source = next(iter(deformation(Path([SaddleConnection(S, source)])).value().tighten()))

In [ ]:
for component in cylinders:
    for connection in component.right():
        connection = connection.saddleConnection()
        from pyflatsurf import flatsurf
        if flatsurf.Vertex.source(source.source(), component.decomposition().surface().combinatorial()) != flatsurf.Vertex.source(connection.source(), component.decomposition().surface().combinatorial()):
            continue
        if source.ccw(connection) != flatsurf.CCW.CLOCKWISE and source.angle(connection) == 0:
            break
    else:
        continue
    break
else:
    assert(False)

print(component)
WW.flow_components = [component]

## Passing to a Delaunay Triangulation
To make the picture easier to understand, we now pass to a Delaunay triangulation of the deformed surface.

In principle, it would be enough to just call `deformed.delaunay()`. However, we want to also construct a morphism between the two triangulation so that we can make sense of the flow component in the new triangulation.

In [ ]:
delaunay = deformed.clone()

In [ ]:
from pyflatsurf import flatsurf
trivial = flatsurf.Deformation[type(delaunay)](delaunay.clone())
retriangulation = flatsurf.Tracked(delaunay.combinatorial(), deformation)

In [ ]:
delaunay.delaunay()

In [ ]:
import cppyy

In [ ]:
retriangulation = retriangulation.codomain().isomorphism(delaunay).value() * retriangulation.value()

In [ ]:
source = list(retriangulation(source).value().tighten())[0]

In [ ]:
WWW = Widget(delaunay)
WWW

In [ ]:
O = GL2ROrbitClosure(delaunay)
D = O.decomposition(direction.vector())
print(D)

In [ ]:
cylinders = [component for component in D.components() if component.cylinder()]

In [ ]:
WWW.flow_components = cylinders

In [ ]:
for component in cylinders:
    for connection in component.right():
        connection = connection.saddleConnection()
        from pyflatsurf import flatsurf
        if flatsurf.Vertex.source(source.source(), component.decomposition().surface().combinatorial()) != flatsurf.Vertex.source(connection.source(), component.decomposition().surface().combinatorial()):
            continue
        if source.ccw(connection) != flatsurf.CCW.CLOCKWISE and source.angle(connection) == 0:
            break
    else:
        continue
    break
else:
    assert(False)

print(component)
WWW.flow_components = []
WWW.flow_components = [component]